In [ ]:
import torch

In [ ]:
import math

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
'''
    I borrow these values from the paper.
    In particular, I use one of the std dev they found
        for their natural images dataset
        during the image regression task
        through hyper-parameter sweep.
    I did not redo the hyper-param sweep
    and I don't know if these values are optimal!
'''
channels = 256 #Pg 8
freq = 256 # Pg 8
sig_pos = 6. # Pg 19
sig_gaus = 10. # Pg 19

In [ ]:
'''
    I've modified a few of these funcs so that they take more than just v
    for example, I've unfixed atens/btens in the general form
''' 

In [ ]:
def general(v: torch.Tensor, atens: torch.Tensor, btens: torch.Tensor):
    '''
    This func never gets used.
    Rather, it's the general form that the rest of the funcs take after.
    '''
    muls = btens @ v
    circle = 2 * math.pi * muls
    coses = torch.cos(circle)
    sins = torch.sin(circle)
    scaled_coses = atens * coses
    scaled_sins = atens * sins
    pairs = torch.stack((scaled_coses, scaled_sins), dim=1)
    interleaved = pairs.flatten()
    return interleaved

In [ ]:
'''
    I've also modified the rest of these to be able to take multiple vs
    so that I can compute the feature's for each dataset in one call
'''

In [ ]:
def basic(v: torch.Tensor):
    '''
    I think this basic mapping technically breaks from the general form?
    It doesn't interleave coses/sins, rather it returns big cos/sin blocks.
    '''
    circle = 2 * math.pi * v
    cos = torch.cos(circle)
    sin = torch.sin(circle)
    return torch.cat((cos, sin), dim=-1)

In [ ]:
def positional_encoding(v: torch.Tensor, sigma: float, m: int, dev: torch.device):
    js = torch.arange(m).to(dev)
    fracs = js/m
    sigmas = sigma**fracs
    coeffs = 2 * math.pi * sigmas
    circle = coeffs.unsqueeze(0).unsqueeze(-1) * v.unsqueeze(-2)
    coses = torch.cos(circle)
    sins = torch.sin(circle)
    combine = torch.stack((coses, sins), dim=-2)
    v_groups = combine.view(-1, 2*m, v.size(-1))
    return v_groups

In [ ]:
def gaussian(v: torch.Tensor, sigma: float, m: int, dev: torch.device):
    B = torch.normal(0., sigma, (m, v.size(-1))).to(dev) # mu taken from pg 7
    muls = B @ v.unsqueeze(-1)
    muls = muls.squeeze(-1)
    circle = 2 * math.pi * muls
    coses = torch.cos(circle)
    sins = torch.sin(circle)
    return torch.cat((coses, sins), dim=-1)

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
criterion = nn.MSELoss()

In [ ]:
import torchvision

In [ ]:
vid_dir = "/kaggle/working/"

In [ ]:
!rm -rf {vid_dir}*
!mkdir {vid_dir}nm
!mkdir {vid_dir}basic
!mkdir {vid_dir}pos
!mkdir {vid_dir}gaus

In [ ]:
mona = torchvision.transforms.functional.convert_image_dtype(torchvision.io.read_image("/kaggle/input/monalisaimage/test.jpg"), torch.float32)
inner_color = mona.permute(1,2,0)
xs = torch.linspace(0, 1, inner_color.size(1))
ys = torch.linspace(0, 1, inner_color.size(0))
grid_tuples = torch.meshgrid(xs, ys, indexing='xy')
grid = torch.stack(grid_tuples).permute(1, 2, 0).to(device)
labels = inner_color.reshape(inner_color.size(0) * inner_color.size(1), 3).to(device)

In [ ]:
nm_input = grid.detach().clone().reshape(grid.size(0) * grid.size(1), 2)

In [ ]:
'''
    4 layers with ReLU in between and Sigmoid output,
    Same as used for the image regression task in the paper.
    Though I am not actually reproducing their image regression experiments.
'''
no_map = nn.Sequential(
    nn.Linear(in_features=2, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=3),
    nn.Sigmoid()
).to(device)

optimizer = optim.Adam(no_map.parameters(), lr=10**-2) # Lr Pg 19

In [ ]:
'''
    I think it's a bit weird to call this an epoch?
    But, I really can load the entire single-image dataset
    and do inference on the whole thing
    and do every optim step (of which there is only one)
    in a single go, so it seems, to me, like these are epochs!
'''
for epoch in range(1000):
    epoch_loss = 0.0

    pred = no_map(nm_input)
    loss = criterion(pred, labels)
    
    epoch_loss = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    img = pred.reshape(512,343,3).permute(2,0,1).detach().cpu()
    torchvision.utils.save_image(img, f"{vid_dir}nm/{epoch:04d}_loss_{epoch_loss:.6f}.png")
    print(epoch)
    print(epoch_loss)

In [ ]:
learned = no_map(nm_input.to(device))

In [ ]:
img = learned.view(512, 343, 3).detach().cpu()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img.numpy())
plt.show()

In [ ]:
torchvision.utils.save_image(img.permute(2, 0, 1), "new.png")

In [ ]:
b_input = basic(grid).detach().view(512 * 343, 4)

In [ ]:
basic_map = nn.Sequential(
    nn.Linear(in_features=4, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=3),
    nn.Sigmoid()
).to(device)

optimizer = optim.Adam(basic_map.parameters(), lr=10**-2)

In [ ]:
for epoch in range(1000):
    epoch_loss = 0.0

    pred = basic_map(b_input)
    loss = criterion(pred, labels)
    
    epoch_loss = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    img = pred.reshape(512,343,3).permute(2,0,1).detach().cpu()
    torchvision.utils.save_image(img, f"{vid_dir}basic/{epoch:04d}_loss_{epoch_loss:.6f}.png")
    print(epoch)
    print(epoch_loss)

In [ ]:
learned = basic_map(b_input.to(device))

In [ ]:
img = learned.view(512, 343, 3).detach().cpu()

In [ ]:
plt.imshow(img.numpy())
plt.show()

In [ ]:
pos_input = positional_encoding(grid, sig_pos, freq, device).detach().view(512 * 343, 1024)

In [ ]:
pos_map = nn.Sequential(
    nn.Linear(in_features=1024, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=3),
    nn.Sigmoid()
).to(device)

optimizer = optim.Adam(pos_map.parameters(), lr=10**-3)

In [ ]:
for epoch in range(1000):
    epoch_loss = 0.0

    pred = pos_map(pos_input)
    loss = criterion(pred, labels)
    
    epoch_loss = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    img = pred.reshape(512,343,3).permute(2,0,1).detach().cpu()
    torchvision.utils.save_image(img, f"{vid_dir}pos/{epoch:04d}_loss_{epoch_loss:.6f}.png")
    print(epoch)
    print(epoch_loss)

In [ ]:
learned = pos_map(pos_input.to(device))

In [ ]:
img = learned.view(512, 343, 3).detach().cpu()

In [ ]:
plt.imshow(img.numpy())
plt.show()

In [ ]:
gaus_input = gaussian(grid, sig_gaus, freq, device).detach().view(512 * 343, 512)

In [ ]:
gaus_map = nn.Sequential(
    nn.Linear(in_features=512, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=channels),
    nn.ReLU(),
    nn.Linear(in_features=channels, out_features=3),
    nn.Sigmoid()
).to(device)

optimizer = optim.Adam(gaus_map.parameters(), lr=10**-3)

In [ ]:
for epoch in range(1000):
    epoch_loss = 0.0

    pred = gaus_map(gaus_input)
    loss = criterion(pred, labels)
    
    epoch_loss = loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    img = pred.reshape(512,343,3).permute(2,0,1).detach().cpu()
    torchvision.utils.save_image(img, f"{vid_dir}gaus/{epoch:04d}_loss_{epoch_loss:.6f}.png")
    print(epoch)
    print(epoch_loss)

In [ ]:
learned = gaus_map(gaus_input.to(device))

In [ ]:
img = learned.view(512, 343, 3).detach().cpu()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img.numpy())
plt.show()

In [ ]:
!touch {vid_dir}/test.txt

In [ ]:
!zip -r "{vid_dir}/images.zip" "{vid_dir}" -x "{vid_dir}/images.zip" "{vid_dir}/images.zip/*"